In [16]:
import matplotlib.pyplot as plt
from workflow.scripts.plotting_tools import create_facet_plot
import xarray as xr
from workflow.scripts.utils import calc_relative_change
from workflow.scripts.plotting_tools import global_map
import matplotlib as mpl
import numpy as np

In [3]:
nmodels = len(snakemake.input.path_ctrl)
ctrl_path = snakemake.input.path_ctrl
ctrl_path=sorted(ctrl_path)
exp_path = snakemake.input.path_exp
exp_path=sorted(exp_path)
diffs = {}
for ctrl, exp in zip(ctrl_path, exp_path):
    ds_exp = xr.open_dataset(exp).load()
    ds_ctrl = xr.open_dataset(ctrl).load()
    rel_diff = calc_relative_change(ds_ctrl, ds_exp)
    diffs[ds_exp.source_id] = rel_diff
    

In [20]:
diffs['MPI-ESM-1-2-HAM']['cdnc'].min()

In [33]:
fig,ax = create_facet_plot(nmodels, figsize=(16,10))
cmap = mpl.cm.bwr

colors_under = plt.cm.bwr(np.linspace(.25, 0.50, 256))
colors_above = plt.cm.bwr(np.linspace(0.5, 1, 256))

all_colors = np.vstack((colors_under, colors_above))
offset_cmap = mpl.colors.LinearSegmentedColormap.from_list('offset_bwr',
    all_colors)
# norm = mpl.colors.BoundaryNorm(np.linspace(-100,100,17),17)
norm = mpl.colors.TwoSlopeNorm(vmin=-20., vcenter=0., vmax=100.)
for ds, axi in zip(diffs, ax):
    temp_ds = diffs[ds]
    pcm=temp_ds[temp_ds.variable_id].plot(ax=ax[axi], cmap=offset_cmap, norm=norm,  add_colorbar=False)
    ax[axi].set_title(ds)
    global_map(ax[axi])

cax = fig.add_axes([0.94,0.2,0.02,0.62])
fig.colorbar(pcm, cax=cax, extend='both',
             label='$\Delta$[CDNC] %')
plt.savefig(snakemake.output.outpath, bbox_inches='tight', dpi=144, facecolor='w')
